In [126]:
import numpy as np 
import pandas as pd
import tensorflow as tf
# from tensorflow import keras

In [127]:
train_df = pd.read_csv('../input/digit-recognizer/train.csv')
test_df = pd.read_csv('../input/digit-recognizer/test.csv')

In [128]:
X_train = train_df.drop(columns=['label'])
y_train = train_df['label']
X_test = test_df

In [129]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [130]:
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0],28,28,1)

X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0],28,28,1)

In [131]:
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

In [132]:
from keras.utils.np_utils import to_categorical
n_classes = 10
y_train = to_categorical(y_train, n_classes)

In [133]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [134]:
X_train = np.reshape(X_train, (42000,784,))
X_test = np.reshape(X_test, (28000,784,))

In [135]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

In [136]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten
from keras.models import Model

x1 = Input(shape=(784,))
encoder1 = Dense(512, activation='relu')(x1)
decoder1 = Dense(784, activation='sigmoid')(encoder1)
autoencoder1 = Model(x1, decoder1)
autoencoder1.summary()

In [137]:
autoencoder1.compile(optimizer='adam', loss='mse')
autoencoder1.fit(X_train, X_train, epochs=10)

In [138]:
# input_layer = Input(shape=(784,))
# hidden1 = Dense(units = 512, activation='sigmoid')(input_layer)
# hidden2 = Dense(units = 350, activation='sigmoid')(hidden1)
# hidden3 = Dense(units = 256, activation='sigmoid')(hidden2)
# hidden4 = Dense(units = 128, activation='sigmoid')(hidden3)
# output1 = Dense(units=784, activation='sigmoid')(hidden4)
# autoencoder1 = Model(inputs = input_layer, outputs = output1)
# autoencoder1.summary()
# autoencoder1.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])
# autoencoder1.fit(X_train , X_train , epochs = 15)

In [139]:
autoencoder1_hidden_output = autoencoder1.layers[1].output
trimmed_autoencoder1 = Model(x1,autoencoder1_hidden_output)
X_train1 = trimmed_autoencoder1.predict(X_train)
X_test1 = trimmed_autoencoder1.predict(X_test)

In [140]:
x2 = Input(shape=(512,))
encoder2 = Dense(256, activation='relu')(x2)
decoder2 = Dense(512, activation='sigmoid')(encoder2)
autoencoder2 = Model(x2, decoder2)
autoencoder2.summary()

In [141]:
autoencoder2.compile(optimizer='adam', loss='mse')
autoencoder2.fit(X_train1, X_train1, epochs=50)

In [142]:
autoencoder2_hidden_output = autoencoder2.layers[1].output
trimmed_autoencoder2=Model(x2,autoencoder2_hidden_output)
X_train2 = trimmed_autoencoder2.predict(X_train1)
X_test2 = trimmed_autoencoder2.predict(X_test1)

In [144]:
x3 = Input(shape=(256,))
output = Dense(10, activation='softmax')(x3)
autoencoder3 = Model(x3,output)
autoencoder3.summary()

In [145]:
autoencoder3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
autoencoder3.fit(X_train2, y_train, epochs=50)

In [147]:
predictions = autoencoder3.predict(X_test2)
final_predictions=np.argmax(predictions, axis=1)
print(final_predictions)

In [149]:
submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
submission['label'] = final_predictions
submission.to_csv('submission_csv', index=False)